In [ ]:
# initial populations of network in order A, E, Y 
N_tot = 100 # total
# N is list containing pop of A,E,Y initially
N = [int(A_pop/total_IT_pop*N_tot),int(E_pop/total_IT_pop*N_tot)]
N.append(N_tot-N[0]-N[1])
N

In [ ]:
# creating nodes
G = nx.Graph()
for i in range(N_tot):
    if (i < N[0]):
        age_class = 'A'
    elif(i< N[1]+N[0]):
        age_class = 'E'
    else:
        age_class = 'Y'
    G.add_node(i, age_class = age_class)
color_map = {'A': 'red', 'E': 'green', 'Y': 'yellow'}
node_colors = [color_map[G.nodes[node]['age_class']] for node in G.nodes()]

nx.draw_networkx_nodes(G, pos = nx.spring_layout(G,seed = 0),node_color = node_colors)

In [ ]:
# prob dict 
# average links to sample from by class
avg_links = {
    'A':12,
    'E':4,
    'Y':20
}
# std for such average (sampling)
std_links = {
    'A':3,
    'E':1,
    'Y':3
}

# the initial setup is a FULLY CONNECTED
edges = []
for i in range(len(G.nodes)):
    for j in range(len(G.nodes)):
        if i!=j:
            edges.append((i,j))
G.add_edges_from(edges)
pos = nx.spring_layout(G,seed = 0)
nx.draw_networkx_nodes(G, pos = pos,node_color = node_colors)
nx.draw_networkx_edges(G, pos=pos)

# now we go with Barabasi Albert
for i in range(N, 4000):
    G.add_node(i)
    n_links

In [ ]:
# prob dict 
# average links to sample from by class
avg_links = {
    'A': 12,
    'E': 4,
    'Y': 20
}
# std for such average (sampling)
std_links = {
    'A': 3,
    'E': 1,
    'Y': 3
}

ages=np.array(['A','E','Y'])

# now we go with Barabasi Albert
final_size = 4000  # Define the final network size

for i in range(N_tot, final_size):
    # Randomly assign age class to new node based on population distribution
    r = np.random.uniform(0,1)
    if r < N[0]/N_tot:
        age_class = 'A'
    elif r < (N[0]+N[1])/N_tot:
        age_class = 'E'
    else:
        age_class = 'Y'
    
    # Add the new node with its age class
    G.add_node(i, age_class=age_class)
    
    # Sample number of links from normal distribution based on age class
    n_links = max(1, int(np.random.normal(avg_links[age_class], std_links[age_class])))

    
    link=0
    connected_nodes=[]
    if age_class=='A':
        idx=0
    elif age_class=='E':
        idx=1
    else:
        idx=2
        
    while link < n_links:
        # select an age group based on contact probabilities
        target_age_probs = norm_contacts_matrix[idx,:]
        target_age = np.random.choice(ages, p=target_age_probs)
        
        # Get all nodes of the target age
        target_nodes = [node for node in G.nodes if G.nodes[node]['age_class'] == target_age]

        # degree of each potential target node for linking probability
        K = [G.degree(node) for node in target_nodes]
        total_degree = sum(K)

        link_prob = K/total_degree  ###linking probability for each target node

        while True:  ##check to see if I already connected 
            chosen_node = np.random.choice(target_nodes, p=link_prob)
            if chosen_node not in connected_nodes:
                linked_node = chosen_node
                connected_nodes.append(linked_node)
                break
        # Add edge
        G.add_edge(i, linked_node)
        link += 1
    